In [1]:
# Import our dependencies
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

data = Path('Resourses/patient_data.csv')
df = pd.read_csv(data)
df.head()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,...,glyburide-metformin:Up,A1Cresult:>7,A1Cresult:>8,A1Cresult:None,A1Cresult:Norm,max_glu_serum:>200,max_glu_serum:>300,max_glu_serum:None,max_glu_serum:Norm,readmitted
0,4,66,3,18,0,0,1,9,1,1,...,0,0,0,1,0,0,0,1,0,1
1,2,48,0,15,4,0,0,7,0,0,...,0,0,0,1,0,0,0,1,0,0
2,4,21,3,23,1,0,2,7,0,1,...,0,0,0,1,0,0,0,1,0,0
3,5,38,0,5,0,0,0,2,1,1,...,0,0,1,0,0,0,0,1,0,0
4,1,6,0,6,0,0,0,6,0,1,...,0,0,0,1,0,0,0,1,0,1


In [2]:
# Split target column from dataset
y = df['readmitted']
X = df.drop(columns='readmitted')

In [3]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create scaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train.columns)
hidden_layer1 = 8
hidden_layer2 = 8
hidden_layer3 = 8

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Input(shape=[input_features]))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 8)                   │           1,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,361 (5.32 KB)

 Trainable params: 1,361 (5.32 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "precision", "recall"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 4s 875us/step - accuracy: 0.5668 - loss: 0.6773 - precision: 0.5498 - recall: 0.3915
Epoch 2/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 862us/step - accuracy: 0.6310 - loss: 0.6379 - precision: 0.6287 - recall: 0.4862
Epoch 3/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 799us/step - accuracy: 0.6348 - loss: 0.6371 - precision: 0.6297 - recall: 0.4914
Epoch 4/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 834us/step - accuracy: 0.6385 - loss: 0.6345 - precision: 0.6322 - recall: 0.5257
Epoch 5/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 812us/step - accuracy: 0.6380 - loss: 0.6339 - precision: 0.6290 - recall: 0.5165
Epoch 6/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 867us/step - accuracy: 0.6382 - loss: 0.6328 - precision: 0.6326 - recall: 0.5266
Epoch 7/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 801us/step - accuracy: 0.6419 - loss: 0.6308 - precision: 0.6332 - recall: 0.5296
Epoch 8/100
2386/2386 ━━━━━━━━━━━━━━━━━━━━ 2s 860us/step - accuracy: 0.6426 - loss: 0.6289 - precision: 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_precision, model_recall = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Accuracy: {model_accuracy}, Loss: {model_loss}, Precision: {model_precision}, Recall: {model_recall} ")

796/796 - 1s - 1ms/step - accuracy: 0.6194 - loss: 0.6520 - precision: 0.6052 - recall: 0.5014
Accuracy: 0.6194332242012024, Loss: 0.6519728899002075, Precision: 0.6052089929580688, Recall: 0.5013644695281982 


In [11]:
# Export our model to HDF5 file
nn.save("NN_1.h5")